In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from MDAnalysis import Universe
from MDAnalysis.analysis.align import rotation_matrix
from MDAnalysis.analysis.psa import PSA
from psa_identifier import PSAIdentifier

In [3]:
u_I = Universe('structs/2X79_ca_aligned.pdb')
u_O = Universe('structs/2JLN_ca_aligned.pdb')
ca_I = u_I.select_atoms('name CA')
ca_O = u_O.select_atoms('name CA')

In [4]:
u_I.atoms.translate(-ca_I.select_atoms('name CA').center_of_mass())
u_O.atoms.translate(-ca_O.select_atoms('name CA').center_of_mass())

array([-0.00046436, -0.00017927,  0.00023112])

In [5]:
I_ca_coords = ca_I.select_atoms('name CA').positions
O_ca_coords = ca_O.select_atoms('name CA').positions

In [6]:
R, rmsd_value = rotation_matrix(I_ca_coords, O_ca_coords)

In [7]:
u_O.atoms.rotate(R)

matrix([[  1.00000000e+00,  -1.09714279e-06,   1.42092279e-06],
        [  1.09714370e-06,   1.00000000e+00,  -6.34948890e-07],
        [ -1.42092210e-06,   6.34950482e-07,   1.00000000e+00]], dtype=float32)

In [8]:
reference_coordinates = 0.5*(ca_I.select_atoms('name CA').positions
    + ca_O.select_atoms('name CA').positions)

In [9]:
u_ref = Universe('structs/2X79_ca_aligned.pdb')
u_ref.atoms.translate(-u_ref.select_atoms('name CA').CA.center_of_mass())
u_ref.select_atoms('name CA').CA.set_positions(reference_coordinates)

In [10]:
method_names = ['GOdMD']
labels = [] # Heat map labels
simulations = [] # List of simulation topology/trajectory filename pairs
universes = [] # List of MDAnalysis Universes representing simulations

In [11]:
for method in method_names:
    # Note: DIMS uses the PSF topology format
    topname = 'top.psf' if 'DIMS' in method or 'TMD' in method else 'top.pdb'
    pathname = 'fitted_psa.dcd'
    method_dir = 'methods/{}'.format(method)
    if method is not 'LinInt':
        for run in xrange(1, 4): # 3 runs per method
            run_dir = '{}/{:03n}'.format(method_dir, run)
            topology = '{}/{}'.format(method_dir, topname)
            trajectory = '{}/{}'.format(run_dir, pathname)
            labels.append(method + '(' + str(run) + ')')
            simulations.append((topology, trajectory))
    else: # only one LinInt trajectory
        topology = '{}/{}'.format(method_dir, topname)
        trajectory = '{}/{}'.format(method_dir, pathname)
        labels.append(method)
        simulations.append((topology, trajectory))

In [12]:
for sim in simulations:
    universes.append(Universe(*sim))

ValueError: The topology and DCD trajectory files don't have the same number of atoms!
Topology number of atoms 463
Trajectory: methods/GOdMD/001/fitted_psa.dcd Number of atoms 425